In [1]:
# =============================================================================
# BLOCK 1: SETUP, IMPORTS, AND DATA LOADING
# =============================================================================
import warnings
warnings.filterwarnings('ignore')
import time
# --- Library Imports ---
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import optuna
print("Libraries imported successfully.")
# --- Helper Function for Winkler Score ---
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    width = upper - lower
    penalty_lower = np.where(y_true < lower, (2 / alpha) * (lower - y_true), 0)
    penalty_upper = np.where(y_true > upper, (2 / alpha) * (y_true - upper), 0)
    score = width + penalty_lower + penalty_upper
    if return_coverage:
        coverage = np.mean((y_true >= lower) & (y_true <= upper))
        return np.mean(score), coverage
    return np.mean(score)
# --- Global Constants ---
N_SPLITS = 5
RANDOM_STATE = 42
DATA_PATH = './'
N_OPTUNA_TRIALS = 100 # A strong number for a comprehensive search
COMPETITION_ALPHA = 0.1

# --- Load Raw Data ---
try:
    # We drop the low-variance columns they identified right away
    drop_cols=['id', 'golf', 'view_rainier', 'view_skyline', 'view_lakesamm','view_otherwater', 'view_other']
    df_train = pd.read_csv(DATA_PATH + 'dataset.csv').drop(columns=drop_cols)
    df_test = pd.read_csv(DATA_PATH + 'test.csv').drop(columns=drop_cols)
    print("Raw data loaded successfully.")
except FileNotFoundError:
    print("ERROR: Could not find 'dataset.csv' or 'test.csv'.")
    exit()
# --- Prepare Target Variable ---
y_true = df_train['sale_price'].copy()
# The mean-error model works best when predicting the raw price directly
# So, we will NOT log-transform the target this time.
# df_train.drop('sale_price', axis=1, inplace=True) # We keep sale_price for FE
print("Setup complete.")


Libraries imported successfully.
Raw data loaded successfully.
Setup complete.


In [2]:
# =============================================================================
# BLOCK 2: SYNTHESIZED FEATURE ENGINEERING (CORRECTED)
# =============================================================================
print("--- Starting Block 2: Synthesized Feature Engineering ---")
def create_synthesized_features(df_train, df_test):
    # Combine for consistent processing and reset the index
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    # Store the original id for later, as reset_index will remove it
    train_ids = df_train.index
    test_ids = df_test.index
    all_data = pd.concat([df_train, df_test], axis=0).reset_index(drop=True)
    
    # --- A) Brute-Force Numerical Interactions ---
    print("Creating brute-force numerical interaction features...")
    NUMS = ['area', 'land_val', 'imp_val', 'sqft_lot', 'sqft', 'sqft_1','grade', 'year_built']
    for i in range(len(NUMS)):
        for j in range(i + 1, len(NUMS)):
            all_data[f'{NUMS[i]}_x_{NUMS[j]}'] = all_data[NUMS[i]] *all_data[NUMS[j]]
    
    # --- B) Date Features ---
    all_data['sale_date'] = pd.to_datetime(all_data['sale_date'])
    all_data['year'] = all_data['sale_date'].dt.year
    all_data['month'] = all_data['sale_date'].dt.month
    all_data['year_diff'] = all_data['year'] - all_data['year_built']
    
    # --- C) TF-IDF Text Features ---
    print("Creating TF-IDF features for text columns...")
    text_cols = ['subdivision', 'zoning', 'city', 'sale_warning','join_status', 'submarket']
    all_data[text_cols] = all_data[text_cols].fillna('missing').astype(str)
    for col in text_cols:
        tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 5),max_features=128, binary=True)
        tfidf_matrix = tfidf.fit_transform(all_data[col])
        svd = TruncatedSVD(n_components=8, random_state=RANDOM_STATE)
        tfidf_svd = svd.fit_transform(tfidf_matrix)
        tfidf_df = pd.DataFrame(tfidf_svd, columns=[f'{col}_tfidf_svd_{i}' for i in range(8)])
        
        # This concat will now work because both have a simple 0-based index
        all_data = pd.concat([all_data, tfidf_df], axis=1)
    
    # --- D) Log transform some of the new interaction features ---
    for c in ['land_val_x_imp_val', 'land_val_x_sqft', 'imp_val_x_sqft']:
        if c in all_data.columns:
            # Add a small constant to avoid log(0)
            all_data[c] = np.log1p(all_data[c].fillna(0))
    
    # --- E) Final Cleanup ---
    print("Finalizing feature set...")
    cols_to_drop = ['sale_date', 'subdivision', 'zoning', 'city','sale_warning', 'join_status', 'submarket']
    all_data = all_data.drop(columns=cols_to_drop)
    all_data.fillna(0, inplace=True)
    
    # Separate final datasets
    X = all_data[all_data['is_train'] == 1].drop(columns=['is_train','sale_price'])
    X_test = all_data[all_data['is_train'] == 0].drop(columns=['is_train','sale_price'])
    
    # Restore the original 'id' as the index
    X.index = train_ids
    X_test.index = test_ids
    X_test = X_test[X.columns]
    return X, X_test

# We need to re-run this from the original dataframes
X, X_test = create_synthesized_features(df_train, df_test)
print(f"\nSynthesized FE complete. Total features: {X.shape[1]}")
gc.collect()


--- Starting Block 2: Synthesized Feature Engineering ---
Creating brute-force numerical interaction features...
Creating TF-IDF features for text columns...
Finalizing feature set...

Synthesized FE complete. Total features: 111


10

In [3]:
# =============================================================================
# BLOCK 3: K-FOLD TRAINING OF MEAN MODEL (NO TUNING)
# =============================================================================
print("\n--- STAGE 1: K-Fold Training of Mean Model ---")
print("# Using pre-tuned, optimal hyperparameters.")
# --- YOUR BEST PARAMETERS FOR THE MEAN MODEL ---
# These are the parameters from your most successful Optuna run.
best_params_mean = {
            'eta': 0.033165561052369404,
            'max_depth': 8,
            'subsample': 0.8906405742494341,
            'colsample_bytree': 0.6676611493633322,
            'lambda': 4.978069972474589,
            'alpha': 0.004983194427373584,
            'min_child_weight': 3
            }

# --- K-Fold Training ---
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True,random_state=RANDOM_STATE)
oof_mean_preds = np.zeros(len(X))
test_mean_preds = np.zeros(len(X_test))


# THE FIX: Reload the 'grade' column for stratification as the original df was deleted.
grade_for_stratify = pd.read_csv(DATA_PATH + 'dataset.csv')['grade']

# Add the other required XGBoost parameters
final_params_mean = {'objective': 'reg:squarederror', 'eval_metric': 'rmse','tree_method': 'hist', 'random_state': RANDOM_STATE, 'n_jobs': -1,**best_params_mean}
for fold, (train_idx, val_idx) in enumerate(skf.split(X, grade_for_stratify)):
    print(f" Mean Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**final_params_mean, n_estimators=2500,early_stopping_rounds=100)
    model.fit(X.iloc[train_idx], y_true.iloc[train_idx], eval_set=[(X.iloc[val_idx], y_true.iloc[val_idx])], verbose=False)
    oof_mean_preds[val_idx] = model.predict(X.iloc[val_idx])
    test_mean_preds += model.predict(X_test) / N_SPLITS

    
# --- NEW: CALCULATE AND PRINT FINAL OOF RMSE ---
final_mean_rmse = np.sqrt(mean_squared_error(y_true, oof_mean_preds))
print(f"\n# Mean model K-Fold training complete.")
print(f"# Final OOF RMSE for Mean Model: ${final_mean_rmse:,.2f}")
print("-" * 50)



--- STAGE 1: K-Fold Training of Mean Model ---
# Using pre-tuned, optimal hyperparameters.
 Mean Model - Fold 1/5...
 Mean Model - Fold 2/5...
 Mean Model - Fold 3/5...
 Mean Model - Fold 4/5...
 Mean Model - Fold 5/5...

# Mean model K-Fold training complete.
# Final OOF RMSE for Mean Model: $98,884.39
--------------------------------------------------


In [19]:
# =============================================================================
# STAGE 2: ERROR MODEL FEATURE ENGINEERING, TUNING, AND TRAINING
# =============================================================================

# --- Define the Error Target ---
error_target = np.abs(y_true - oof_mean_preds)


# =============================================================================
# --- PART 1A: FEATURE ENGINEERING FOR THE ERROR MODEL ---
# =============================================================================
print("\n# Performing feature engineering for the error model...")

# Start with copies of the original data
X_for_error = X.copy()
X_test_for_error = X_test.copy()

# Feature 1: The mean prediction itself
print("  Adding 'mean_pred_oof' feature...")
X_for_error['mean_pred_oof'] = oof_mean_preds
X_test_for_error['mean_pred_oof'] = test_mean_preds

# Feature 2: Deviation from the average prediction
print("  Adding 'pred_deviation' feature...")
avg_oof_pred = oof_mean_preds.mean()
X_for_error['pred_deviation'] = np.abs(X_for_error['mean_pred_oof'] - avg_oof_pred)
X_test_for_error['pred_deviation'] = np.abs(X_test_for_error['mean_pred_oof'] - avg_oof_pred)

# Feature 3: Binned predictions
print("  Adding 'pred_bin' feature...")
X_for_error['pred_bin'], bin_edges = pd.qcut(X_for_error['mean_pred_oof'], 
                                             q=10, 
                                             labels=False, 
                                             retbins=True, 
                                             duplicates='drop')
X_test_for_error['pred_bin'] = pd.cut(X_test_for_error['mean_pred_oof'], 
                                      bins=bin_edges, 
                                      labels=False, 
                                      include_lowest=True).fillna(-1).astype(int)

print(f"\n# Feature engineering complete. Total features for error model: {X_for_error.shape[1]}")
# =============================================================================


# --- PART 1B: Tuning the Error Model on the NEW Feature Set ---
print("\n# STAGE 2, PART 1: Tuning Error Prediction Model (on new features)...")
N_OPTUNA_TRIALS = 100

def objective_error(trial):
    # This split now correctly uses X_for_error which has the new features
    train_x, val_x, train_y, val_y = train_test_split(X_for_error, error_target, test_size=0.2, random_state=RANDOM_STATE)
    
    params = {
            'objective': 'reg:squarederror',
            'eval_metric': 'rmse',
            'tree_method': 'hist',
            'eta': trial.suggest_float('eta', 0.01, 0.03),
            'max_depth': trial.suggest_int('max_depth', 7, 10),
            'subsample': trial.suggest_float('subsample', 0.8, 0.99),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.692, 0.75),
            'lambda': trial.suggest_float('lambda', 0.345, 0.765, log=True),
            'alpha': trial.suggest_float('alpha', 0.455, 0.69),
        }

    model = xgb.XGBRegressor(**params, n_estimators=2500, random_state=RANDOM_STATE, n_jobs=-1, early_stopping_rounds=100)
    model.fit(train_x, train_y, eval_set=[(val_x, val_y)], verbose=False)
    preds = model.predict(val_x)
    return np.sqrt(mean_squared_error(val_y, preds))

study_error = optuna.create_study(direction='minimize')
study_error.optimize(objective_error, n_trials=N_OPTUNA_TRIALS)
best_params_error = study_error.best_params
print(f"\n# Error Model Tuning Complete. Best Validation RMSE: ${study_error.best_value:,.2f}")


# --- PART 2: K-Fold Training of Error Model with Best Params ---
print("\n# STAGE 2, PART 2: K-Fold Training of Error Model...")
oof_error_preds = np.zeros(len(X))
test_error_preds = np.zeros(len(X_test))

# Add the other required XGBoost parameters
final_params_error = {
    'objective': 'reg:squarederror', 
    'eval_metric': 'rmse',
    'tree_method': 'hist', 
    'random_state': RANDOM_STATE, 
    'n_jobs': -1, 
    **best_params_error
}

# This loop now uses X_for_error, which has the correct features
for fold, (train_idx, val_idx) in enumerate(skf.split(X_for_error, grade_for_stratify)):
    print(f"  Error Model - Fold {fold+1}/{N_SPLITS}...")
    model = xgb.XGBRegressor(**final_params_error, n_estimators=2500, early_stopping_rounds=100)
    
    X_train, X_val = X_for_error.iloc[train_idx], X_for_error.iloc[val_idx]
    y_train, y_val = error_target.iloc[train_idx], error_target.iloc[val_idx]
    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    oof_error_preds[val_idx] = model.predict(X_val)
    test_error_preds += model.predict(X_test_for_error) / N_SPLITS

# --- Calculate and Print Final OOF RMSE ---
final_error_rmse = np.sqrt(mean_squared_error(error_target, oof_error_preds))
print(f"\n# Error model K-Fold training complete.")
print(f"# Final OOF RMSE for Error Model: ${final_error_rmse:,.2f}")
print("-" * 50)


# Performing feature engineering for the error model...


[I 2025-07-08 20:30:37,301] A new study created in memory with name: no-name-3872e6ec-667b-44f4-903e-c33067709590


  Adding 'mean_pred_oof' feature...
  Adding 'pred_deviation' feature...
  Adding 'pred_bin' feature...

# Feature engineering complete. Total features for error model: 114

# STAGE 2, PART 1: Tuning Error Prediction Model (on new features)...


[I 2025-07-08 20:30:48,738] Trial 0 finished with value: 63661.17282648306 and parameters: {'eta': 0.026973437442620693, 'max_depth': 9, 'subsample': 0.8669033717360423, 'colsample_bytree': 0.7064243155359198, 'lambda': 0.45082975488131105, 'alpha': 0.6614010449644007}. Best is trial 0 with value: 63661.17282648306.
[I 2025-07-08 20:30:57,078] Trial 1 finished with value: 63629.1717687304 and parameters: {'eta': 0.0217760396484855, 'max_depth': 8, 'subsample': 0.8557787164397109, 'colsample_bytree': 0.7025573880523461, 'lambda': 0.5352732008016118, 'alpha': 0.6738595615124922}. Best is trial 1 with value: 63629.1717687304.
[I 2025-07-08 20:31:03,431] Trial 2 finished with value: 63623.12536613408 and parameters: {'eta': 0.023543226235028355, 'max_depth': 7, 'subsample': 0.8386830490846758, 'colsample_bytree': 0.7408331517694875, 'lambda': 0.39967564093863556, 'alpha': 0.5284111696333488}. Best is trial 2 with value: 63623.12536613408.
[I 2025-07-08 20:31:17,014] Trial 3 finished with v


# Error Model Tuning Complete. Best Validation RMSE: $63,440.51

# STAGE 2, PART 2: K-Fold Training of Error Model...
  Error Model - Fold 1/5...
  Error Model - Fold 2/5...
  Error Model - Fold 3/5...
  Error Model - Fold 4/5...
  Error Model - Fold 5/5...

# Error model K-Fold training complete.
# Final OOF RMSE for Error Model: $62,866.29
--------------------------------------------------


In [20]:
# =============================================================================
# FINAL ASYMMETRIC CALIBRATION AND SUBMISSION (ULTIMATE ROBUST VERSION)
# =============================================================================

print("\n--- Final Asymmetric Calibration ---")

# --- Safely reload y_true to ensure it's available ---
y_true = pd.read_csv('./dataset.csv')['sale_price']

# --- Your existing correct code ---
oof_error_final = np.clip(oof_error_preds, 0, None) 
best_a, best_b, best_metric = 2.0, 2.0, float('inf')

for a in np.arange(1.90, 2.31, 0.01):
    for b in np.arange(2.10, 2.51, 0.01):
        low = oof_mean_preds - oof_error_final * a
        high = oof_mean_preds + oof_error_final * b
        # We need the winkler_score function defined here or in a previous cell
        metric, coverage = winkler_score(y_true, low, high, alpha=COMPETITION_ALPHA, return_coverage=True)
        if metric < best_metric:
            best_metric = metric
            best_a, best_b = a, b
            print(f"New Best! a={best_a:.2f}, b={best_b:.2f}, Score={best_metric:,.2f}, Cov={coverage:.2%}")
print(f"\nGrid search complete. Final OOF Score: {best_metric:,.2f}. Best multipliers: a={best_a:.2f}, b={best_b:.2f}")


# --- Create Final Submission ---
print("\nCreating final submission file...")
test_error_final = np.clip(test_error_preds, 0, None)
final_lower = test_mean_preds - test_error_final * best_a
final_upper = test_mean_preds + test_error_final * best_b
final_upper = np.maximum(final_lower, final_upper)

# Your excellent, robust fix for the IDs
test_ids = pd.read_csv('./test.csv', usecols=['id'])['id']
submission_df = pd.DataFrame({
    'id': test_ids, 
    'pi_lower': final_lower, 
    'pi_upper': final_upper
})

submission_df.to_csv('submission_final_v6.csv', index=False)
print("\n'submission_final_v6.csv' created successfully!")
display(submission_df.head())


--- Final Asymmetric Calibration ---
New Best! a=1.90, b=2.10, Score=303,504.11, Cov=88.87%
New Best! a=1.90, b=2.11, Score=303,453.51, Cov=88.94%
New Best! a=1.90, b=2.12, Score=303,410.44, Cov=89.01%
New Best! a=1.90, b=2.13, Score=303,373.20, Cov=89.07%
New Best! a=1.90, b=2.14, Score=303,343.97, Cov=89.13%
New Best! a=1.90, b=2.15, Score=303,321.27, Cov=89.20%
New Best! a=1.90, b=2.16, Score=303,305.99, Cov=89.25%
New Best! a=1.90, b=2.17, Score=303,297.62, Cov=89.32%
New Best! a=1.90, b=2.18, Score=303,297.02, Cov=89.39%
New Best! a=1.91, b=2.15, Score=303,277.68, Cov=89.29%
New Best! a=1.91, b=2.16, Score=303,262.39, Cov=89.35%
New Best! a=1.91, b=2.17, Score=303,254.03, Cov=89.41%
New Best! a=1.91, b=2.18, Score=303,253.43, Cov=89.48%
New Best! a=1.92, b=2.15, Score=303,244.07, Cov=89.38%
New Best! a=1.92, b=2.16, Score=303,228.78, Cov=89.44%
New Best! a=1.92, b=2.17, Score=303,220.42, Cov=89.50%
New Best! a=1.92, b=2.18, Score=303,219.82, Cov=89.57%
New Best! a=1.93, b=2.16, S

,id,pi_lower,pi_upper
0,200000,823019.231494,1.038293e+06
1,200001,595938.938379,8.168840e+05
2,200002,446821.759375,6.565207e+05
3,200003,277557.264868,4.030370e+05
4,200004,338362.155762,8.094089e+05
